In [1]:
using LinearAlgebra
using HDF5

In [2]:
function basisPartition(wf, sysIndx, nSites)

    nSys = size(sysIndx)[1]
    envIndx = [i for i = 1:nSites]
    for i = 1:nSys
        filter!(e->e≠sysIndx[i], envIndx)
    end
    nEnv = size(envIndx)[1]
    
    @assert nSys + nEnv == nSites
    
    dimDof = 2
    dimHilSys = dimDof^nSys
    dimHilEnv = dimDof^nEnv

    sysBitConfig = zeros(Bool, (dimHilSys, nSites))
    envBitConfig = zeros(Bool, (dimHilEnv, nSites))

    axisScalar = ones(UInt32, nSites)  # UInt32: upto 2^32 (32 sites ED)
    for i = 1 : nSites
        axisScalar[nSites + 1 - i] *= dimDof^(i - 1)
    end

    for i = 1 : dimDof^nSys
        upperpad::Int = size(digits(dimDof^(nSys - 1), base = 2))[1]
        exBits = digits(i-1, base = 2, pad = upperpad)  # expose bits' position for sys dofs

        for j = 1 : size(sysIndx)[1]
            sysBitConfig[i, sysIndx[j]] = exBits[j]
        end

    end

    for i = 1 : dimDof^nEnv
        upperpad::Int = size(digits(dimDof^(nEnv - 1), base = 2))[1]
        exBits = digits(i-1, base = 2, pad = upperpad)  # expose bits' position for evn dofs

        for j = 1 : size(envIndx)[1]
            envBitConfig[i, envIndx[j]] = exBits[j]
        end

    end


    @assert dimHilSys == size(sysBitConfig)[1]
    @assert dimHilEnv == size(envBitConfig)[1]


    # Index matrix
    matIndx = zeros(UInt32, (dimHilSys, dimHilEnv))    # UInt32: upto 2^32 (32 sites ED)

    for i = 1 : dimHilSys
        for j = 1 : dimHilEnv
            matIndx[i,j] = sum((sysBitConfig[i,:] + envBitConfig[j,:]).* axisScalar) + 1 # +1 is because 1-based
        end
    end
    
    # wavefunction as matrix
    wf_as_mat = zeros(Complex{Float64}, (dimHilSys, dimHilEnv))
    
    for i = 1 : dimHilSys
        for j = 1 : dimHilEnv
            wf_as_mat[i,j] = wf[matIndx[i,j]]
        end
    end
    
    return wf_as_mat
end

basisPartition (generic function with 1 method)

In [3]:
# For continuous index partition
hd5 = h5open("../dataSpec.hdf5","r")
dset=hd5["3.Eigen/Wavefunctions"]
gs=read(dset)
close(hd5)

ee = Float32[]
sysIndx = Int8[]
for i = 1 : 9
    push!(sysIndx, i)
    wf_as_mat = basisPartition(gs, sysIndx, 18)
    
    rdm = wf_as_mat * adjoint(wf_as_mat)
    entS = eigvals(rdm)
    topop = count(i->(i<0), entS)
    PentS = entS[topop+1:end]  # popped out negative values

     vN = - transpose(PentS) * log.(PentS)
    push!(ee, vN)
end

@show ee

ee = Float32[0.5901132, 0.75179315, 0.90519303, 1.0285116, 1.109273, 1.126989, 1.1446508, 1.1407921, 1.154656]


9-element Array{Float32,1}:
 0.5901132
 0.75179315
 0.90519303
 1.0285116
 1.109273
 1.126989
 1.1446508
 1.1407921
 1.154656

In [6]:
# For discrete index partition
hd5 = h5open("../dataSpec.hdf5","r")
dset=hd5["3.Eigen/Wavefunctions"]
evec=read(dset)
sysIndx = Int8[1]
@time wf_as_mat = basisPartition(evec, sysIndx, 20)
@show size(wf_as_mat);
close(hd5)

  0.509682 seconds (5.24 M allocations: 974.015 MiB, 22.78% gc time)
size(wf_as_mat) = (2, 524288)


In [7]:
# then trace out enironment
rdm = wf_as_mat * adjoint(wf_as_mat)
entS = eigvals(rdm)
topop = count(i->(i<0), entS)
PentS = entS[topop+1:end]  # popped out negative values
- transpose(PentS) * log.(PentS)

0.6750665976888718